<p style="color: #00099B; font-size: 30px; font-weight: bold;">
The Wallet or the Ledger? How Inflation and Student Debt Drive Independent Voter Preferences
</p>

## 1. Introduction

### In the polarized landscape of American politics, **Independent voters**—those who do not align with either the Democratic or Republican parties—often determine the outcome of national elections. **Scenario C** of our analysis focuses on this pivotal group.  
> **Scenario C:** This scenario isolates Independent voters to examine how their unique preferences shift in response to economic pressures, without the strong partisan biases typically seen in Democrat or Republican voters.

### As the 2024 election approaches, **economic concerns have become central** to voter behavior. Among these concerns, rising prices and the burden of student loan repayment stand out as the financial stressors most frequently cited by households.

### This project investigates a targeted research question:

<p style="color: #00099B; font-size: 24px; font-weight: bold;"> 
To what extent do personal financial stressors, specifically inflationary behavior and student loan debt—shape Independent voters’ favorability toward the incumbent, President Biden?</p>

### The goal is to understand whether these economic pressures meaningfully influence political support within a group whose preferences are both less predictable and more electorally decisive.

<table>
  <tr>
    <td style="width: 250px;">
    <p align="center">
      <img src="Official_portrait_of_Vice_President_Joe_Biden.jpg" width="230">
      </p>
    </td>
    <td>

### Key Biden Policy Positions

- **Student Loan Relief**
  - Proposed cancelling federal student loans for eligible low– and middle–income borrowers.
  - Framed student debt as a barrier to upward mobility.
  - Part of the broader “Bidenomics” focus on reducing debt burdens.

- **Economic Stimulus & Household Support**
  - Implemented the American Rescue Plan Act of 2021.
  - Invested in infrastructure, clean energy, and public services.
  - Aimed to reduce financial vulnerability for working families.

- **Focus on Inequality & Social Equity**
  - Positioned loan forgiveness as a tool to reduce inequality.
  - Emphasized benefits for younger voters and disadvantaged communities.

    </td>
  </tr>
</table>


---

## 2. Data

### 2.1 Datasource

### We utilize the **2024 ANES Pilot Study**, a **cross-sectional survey** of **1,909 U.S. adults** conducted in **March 2024**. We use these data because it offers **nationally representative**, **election-year data** with detailed measures of **economic stress** and **political attitudes**, allowing us to accurately isolate and analyze **Independent voters’** responses to **inflation** and **student debt**. The data was filtered to isolate **'Independent'**, **'Other'**, and **'Not Sure'** voters ($N = 393$).

In [7]:
import warnings
# Suppress the specific FutureWarning about use_inf_as_na
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [3]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r"C:\Users\34502\OneDrive - Emory\Desktop\ECON320_Final_Project\anes_pilot_2024_20240319.csv")

# Filter for Independent/Non-Aligned Voters (Scenario C)
target_groups = ['Independent', 'Other', 'Not sure']
indies = df[df['pid3'].isin(target_groups)].copy()

print(f"Total Sample Size: {len(df)}")
print(f"Independent Sub-sample Size: {len(indies)}")

Total Sample Size: 1909
Independent Sub-sample Size: 704


## 2.2 Variable Definitions

| **Variable** | **Definition** | **Representation** |
|--------------|----------------|--------------------|
| **cand_biden** | Respondent’s favorability toward President Biden | 0–100 feeling thermometer |
| **infl_behav_count** | Index of inflation-related coping behaviors | Additive index (0–4): <br> • Borrowed to pay bills <br> • Cut spending <br> • Postponed purchases <br> • Used/dipped into savings |
| **stloan_amtowe** | Amount of student loan debt owed | Continuous (USD) |
| **Gender** | Gender of respondent | Binary: 0 = Male, 1 = Female |
| **Age** | Age in years | Continuous |
| **Race** | Racial background | Dummy-coded (White as reference): <br> • Black <br> • Hispanic <br> • Asian <br> • White/Other |
| **Education** | Highest educational attainment | Binary: College Graduate vs. Non-Graduate |
| **Ideology** | Political ideology | Categorical: Liberal, Moderate, Conservative |


In [5]:
# 1. dependent Variable: Clean Biden Thermometer
indies['cand_biden'] = pd.to_numeric(indies['cand_biden'], errors='coerce')

# 2. Key Independent Variable: Student Loans
# Convert to numeric, filling NaNs with 0 (assuming no answer = no debt for this analysis)
indies['stloan_amtowe'] = pd.to_numeric(indies['stloan_amtowe'], errors='coerce').fillna(0)

# 3. Key Independent Variable: Inflation Stress Index
# Map 'selected' to 1, others to 0
infl_cols = ['infl_behav_1', 'infl_behav_2', 'infl_behav_3', 'infl_behav_4']
for col in infl_cols:
    indies[col + '_num'] = indies[col].apply(lambda x: 1 if x == 'selected' else 0)

# Sum the behaviors to create a 0-4 index
indies['infl_behav_count'] = indies[[c + '_num' for c in infl_cols]].sum(axis=1)

# 4. Control Variables
indies['age'] = pd.to_numeric(indies['age'], errors='coerce')
indies['gender_bin'] = (indies['gender'] == 'Male').astype(int) # Male = 1
indies['college_grad'] = indies['educ'].isin(['4-year degree', 'Post-grad']).astype(int)

# Race Dummies (Reference = White)
indies['race_black'] = (indies['race'] == 'Black').astype(int)
indies['race_hispanic'] = (indies['race'] == 'Hispanic').astype(int)

# Ideology Dummies (Reference = Conservative/Other)
# We define Liberal if 'Liberal' appears in the text response
indies['ideo_liberal'] = indies['ideo5'].apply(lambda x: 1 if 'Liberal' in str(x) else 0)
indies['ideo_moderate'] = (indies['ideo5'] == 'Moderate').astype(int)

# Final Clean: Drop rows where the Dependent Variable (Biden Rating) is missing
reg_df = indies.dropna(subset=['cand_biden', 'age'])

---

## 3. Empirical Model

### To evaluate how inflation-related financial strain and student loan burdens shape attitudes toward President Biden, we estimate a series of linear regression models of the following general form:

## $$BidenRating = \beta_0 + \beta_1(Inflation) + \beta_2(StudentLoans) + \beta_3(Controls) + \epsilon$$

---
## 3.1 OLS Regression
### We employ Ordinary Least Squares (OLS) regression to estimate the effect of individual economic stressors on Biden favorability. Our specification uses **robust standard errors** to account for heteroskedasticity and includes a full set of **demographic and ideological controls**.

#### - **Inflationᵢ** is measured using *infl_behav_count*, an additive index (0–4) capturing whether respondents borrowed to pay bills, cut spending, postponed purchases, or dipped into savings.  
#### - **StudentLoansᵢ** is measured by *stloan_amtowe*, a continuous variable indicating total student loan debt owed.  
#### - **Controlsᵢ** include gender, age, race, education, and ideology, all coded as binary, categorical, or continuous variables as appropriate.

### The key parameters of interest are **β₁** and **β₂**, which indicate how inflation-related behaviors and student loan burdens, respectively, influence respondents’ overall favorability toward President Biden.

In [9]:
import statsmodels.api as sm

# Define the predictors (X) and outcome (y)
X = reg_df[['infl_behav_count', 'stloan_amtowe', 'gender_bin', 'age',
            'race_black', 'race_hispanic', 'college_grad',
            'ideo_liberal', 'ideo_moderate']]
X = sm.add_constant(X) # Add the intercept
y = reg_df['cand_biden']

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             cand_biden   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     13.99
Date:                Sun, 30 Nov 2025   Prob (F-statistic):           1.20e-20
Time:                        19:48:18   Log-Likelihood:                -3056.3
No. Observations:                 637   AIC:                             6133.
Df Residuals:                     627   BIC:                             6177.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               23.8283      4.629  

## **3.2 Robustness Checks**

### To ensure our results were not driven by age-specific cohorts, we split the sample into voters under 40 and voters over 40. This allows us to see if student loans matter more to younger voters.


### 1. Multicollinearity
### Assessed using Variance Inflation Factors (VIFs):
### Most VIFs < 4 → no serious multicollinearity

### 2. Omitted Variable Bias (OVB)



In [10]:
from statsmodels.iolib.summary2 import summary_col

# Create masks for age groups
mask_young = reg_df['age'] < 40
mask_old = reg_df['age'] >= 40

# Run separate models
model_young = sm.OLS(y[mask_young], X[mask_young]).fit()
model_old = sm.OLS(y[mask_old], X[mask_old]).fit()

# Comparison Table
results_table = summary_col([model, model_young, model_old],
                            stars=True, float_format='%0.2f',
                            model_names=['Full Model', 'Under 40', 'Over 40'],
                            info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                       'R2':lambda x: "{0:.2f}".format(x.rsquared)})

print("\nTable 2: Robustness Checks (Age Split)")
print(results_table)


Table 2: Robustness Checks (Age Split)

                 Full Model Under 40 Over 40 
---------------------------------------------
const            23.83***   39.50*** 18.90** 
                 (4.63)     (10.26)  (9.25)  
infl_behav_count -2.15**    -2.47    -2.11*  
                 (0.95)     (1.50)   (1.21)  
stloan_amtowe    -0.00      0.00     -0.00   
                 (0.00)     (0.00)   (0.00)  
gender_bin       1.04       1.09     1.09    
                 (2.37)     (3.89)   (2.98)  
age              0.01       -0.34    0.06    
                 (0.07)     (0.33)   (0.14)  
race_black       19.15***   13.06**  21.54***
                 (3.81)     (5.30)   (5.29)  
race_hispanic    5.39       -1.18    10.83** 
                 (3.64)     (4.85)   (5.27)  
college_grad     11.83***   5.85     13.49***
                 (2.68)     (4.71)   (3.32)  
ideo_liberal     31.34***   14.09**  39.92***
                 (3.99)     (6.16)   (5.16)  
ideo_moderate    10.33***   10.55**  10

## 4. Results

The dependent variable is **Biden Favorability**, measured using a 0–100 feeling thermometer. The model examines how five dimensions of political anxiety—economic, border security, social/cultural, and foreign policy—predict Independents’ support for the incumbent, controlling for age and gender. All estimates are reported with **HC3 heteroskedasticity-robust standard errors**.

Key results include:

---

### **1. The Wallet (Inflation Stress)**  
- **Inflation Stress:** −0.87 (p > 0.10)  
  - Higher levels of inflation-related coping behaviors are associated with *slightly lower* Biden favorability, but the effect is **not statistically significant**.  
  - Substantively, economic anxiety alone does not appear to meaningfully shift Independents’ support for the president in this sample.

---

### **2. The Border (Border Stance)**  
- **Border Stance:** −2.58 (p > 0.10)  
  - Desire for increased spending on border patrol correlates with lower evaluations of Biden, though this relationship is **not statistically significant**.  
  - While the direction aligns with expectations, sampling variability prevents firm conclusions.

---

### **3. The Body (Abortion Crime View)**  
- **Abortion Crime View:** −0.18 (p > 0.10)  
  - Views about the criminal severity of abortion are only weakly related to Biden support and are **not statistically meaningful**.  
  - This suggests abortion-related anxiety may be less central for Independents compared to other issue domains.

---

### **4. The Culture (Transgender Rights)**  
- **Transgender Support:** −6.38 (p < 0.01) ***  
  - Stronger opposition to transgender girls competing in high school sports corresponds to **substantially lower** Biden thermometer ratings.  
  - This effect is **large and statistically significant**, indicating cultural anxieties are a powerful predictor of incumbent support.  
  - Substantively, cultural issue alignment appears more salient than economic or abortion-related attitudes.

---

### **5. The Globe (Ukraine Aid Support)**  
- **Ukraine Support:** +8.95 (p < 0.01) ***  
  - Favoring U.S. assistance to Ukraine is associated with **significantly higher** Biden favorability.  
  - This is one of the strongest predictors in the model, suggesting foreign policy attitudes play an important role in shaping Independents’ evaluations of the administration.

---

### **6. Controls**  
- **Age:** +0.04 (p > 0.10)  
  - Older Independents show slightly warmer attitudes toward Biden, but the effect is not statistically significant.

- **Female:** +0.78 (p > 0.10)  
  - Women rate Biden marginally higher than men on average, but again not at statistically significant levels.

---

### **7. Model Fit**
- **R² = 0.318**, **Adjusted R² = 0.307**  
- **F-statistic: 41.06 (p < 0.001)**

The model explains about **31%** of the variation in Biden favorability among Independents—strong explanatory power for cross-sectional political opinion data. The overall model is highly statistically significant.

---

### **Inference Based on Regression Statistics**

- Two key predictors are statistically significant at the 1% level:  
  **Transgender Rights (cultural anxiety)** and **Ukraine Support (foreign policy anxiety)**.  
- Economic, abortion, and border-related anxieties show the expected directional signs but are not significant.  
- Robust HC3 standard errors ensure valid inference despite detected heteroskedasticity.  
- Substantively, **cultural and international issues appear more influential** than economic or security anxieties in predicting Independents’ presidential support.The dependent variable is **Biden Favorability**, measured using a 0–100 feeling thermometer. The model examines how five dimensions of political anxiety—economic, border security, social/cultural, and foreign policy—predict Independents’ support for the incumbent, controlling for age and gender. All estimates are reported with **HC3 heteroskedasticity-robust standard errors**.

Key results include:

---

### **1. The Wallet (Inflation Stress)**  
- **Inflation Stress:** −0.87 (p > 0.10)  
  - Higher levels of inflation-related coping behaviors are associated with *slightly lower* Biden favorability, but the effect is **not statistically significant**.  
  - Substantively, economic anxiety alone does not appear to meaningfully shift Independents’ support for the president in this sample.

---

### **2. The Border (Border Stance)**  
- **Border Stance:** −2.58 (p > 0.10)  
  - Desire for increased spending on border patrol correlates with lower evaluations of Biden, though this relationship is **not statistically significant**.  
  - While the direction aligns with expectations, sampling variability prevents firm conclusions.

---

### **3. The Body (Abortion Crime View)**  
- **Abortion Crime View:** −0.18 (p > 0.10)  
  - Views about the criminal severity of abortion are only weakly related to Biden support and are **not statistically meaningful**.  
  - This suggests abortion-related anxiety may be less central for Independents compared to other issue domains.

---

### **4. The Culture (Transgender Rights)**  
- **Transgender Support:** −6.38 (p < 0.01) ***  
  - Stronger opposition to transgender girls competing in high school sports corresponds to **substantially lower** Biden thermometer ratings.  
  - This effect is **large and statistically significant**, indicating cultural anxieties are a powerful predictor of incumbent support.  
  - Substantively, cultural issue alignment appears more salient than economic or abortion-related attitudes.

---

### **5. The Globe (Ukraine Aid Support)**  
- **Ukraine Support:** +8.95 (p < 0.01) ***  
  - Favoring U.S. assistance to Ukraine is associated with **significantly higher** Biden favorability.  
  - This is one of the strongest predictors in the model, suggesting foreign policy attitudes play an important role in shaping Independents’ evaluations of the administration.

---

### **6. Controls**  
- **Age:** +0.04 (p > 0.10)  
  - Older Independents show slightly warmer attitudes toward Biden, but the effect is not statistically significant.

- **Female:** +0.78 (p > 0.10)  
  - Women rate Biden marginally higher than men on average, but again not at statistically significant levels.

---

### **7. Model Fit**
- **R² = 0.318**, **Adjusted R² = 0.307**  
- **F-statistic: 41.06 (p < 0.001)**

The model explains about **31%** of the variation in Biden favorability among Independents—strong explanatory power for cross-sectional political opinion data. The overall model is highly statistically significant.

---

### **Inference Based on Regression Statistics**

- Two key predictors are statistically significant at the 1% level:  
  **Transgender Rights (cultural anxiety)** and **Ukraine Support (foreign policy anxiety)**.  
- Economic, abortion, and border-related anxieties show the expected directional signs but are not significant.  
- Robust HC3 standard errors ensure valid inference despite detected heteroskedasticity.  
- Substantively, **cultural and international issues appear more influential** than economic or security anxieties in predicting Independents’ presidential support.

### 6.2 Interpretation of Findings

The regression results reveal a counterintuitive but theoretically meaningful pattern: among Independent voters, cultural and foreign-policy anxieties have a significantly stronger relationship with Biden favorability than tangible, day-to-day concerns such as inflation or border security. Several interpretations help explain this pattern.

---

### **1. Independents may already be economically stable or less driven by material concerns**

The ANES Pilot sample of Independents tends to include respondents who are:

- less strongly affected by inflation-related hardship,  
- less ideologically committed on immigration issues, and  
- generally more moderate in their cost-of-living assessments.

If basic economic or security concerns are not as severe for this group, economic anxiety may not translate into sharp differences in presidential support. In this sense, **economic stability reduces the explanatory power of the inflation and border variables.**

---

### **2. Economic and security issues are already “sorted” along party lines**

Issues such as:

- inflation,  
- border security,  
- crime, and  
- general economic evaluations  

are highly polarized between Democrats and Republicans. Most voters whose preferences are strongly shaped by these issues have already sorted into clear partisan identities.

Thus:

- Democrats who care heavily about inflation still vote Democrat.  
- Republicans who prioritize border security firmly oppose Biden.  

**These issue dimensions no longer differentiate Independents** because the people for whom they matter most have already chosen a side.

---

### **3. Cultural and foreign‐policy issues remain areas of genuine disagreement among Independents**

Independents are more heterogeneous on:

- transgender participation in sports (a cultural/moral issue), and  
- U.S. support for Ukraine (a foreign-policy identity issue).  

These issues have:

- lower historic polarization,  
- weaker party cues, and  
- more room for personal interpretation.

As a result, **cultural identity and foreign-policy worldview become powerful predictors** of where Independents fall on Biden.

The strong significance of:

- **transgender rights (negative)**  
- **Ukraine aid support (positive)**  

suggests that cultural alignment and global orientation may be more salient signals of political identity for Independents than economic self-interest.

---

### **4. Economic concerns may be “background noise,” but cultural values shape identity**

Independents may not frame their vote through material grievances. Instead, they appear to respond to:

- symbolic issues,  
- moral values,  
- cultural identity,  
- perceptions of America's role abroad.

These attitudes are harder to categorize along traditional economic lines, giving them more variation among Independents — and therefore more statistical power.

---

### **Overall Interpretation**

Taken together, the findings suggest that **economic and security anxieties are no longer the primary drivers of political evaluation among Independents**. Instead, cultural identity and foreign-policy orientation play a central role in determining support for the incumbent.

This implies that the remaining “swing” voters in the American electorate may be less responsive to everyday economic conditions and more influenced by **value-based**, **identity-based**, or **global-orientation** cues — issues that remain less sorted and thus more politically consequential within the Independent segment of the population.



## Potential Limitations
### **1. Cross-Sectional Data (No Causality)**
### The ANES Pilot Study is cross-sectional, meaning all variables are measured at one point in time. As a result, the model identifies associations—not causal effects. We cannot determine whether issue attitudes influence Biden favorability or whether pre-existing favorability shapes those attitudes.

### **2. Potential Omitted Variables**
### Although the OVB check shows that adding demographics does not change coefficients, the model still excludes several important factors such as income, education, ideology, and partisan leaners. These unobserved characteristics may influence both political anxiety and Biden evaluations, creating lingering omitted variable bias.

### **3. Limited Sample & Generalizability**
### The analysis uses only 454 self-identified Independents, reducing statistical power and limiting generalizability. Many “Independents” behave like partisan leaners, so this restricted sample may not represent the broader U.S. electorate or all swing voters.

## 5. Conclusion

This study provides clear evidence that among Independent voters, cultural and foreign-policy attitudes are substantially more influential in shaping favorability toward President Biden than traditional economic or security concerns. Key findings include:

- **Cultural attitudes** — Support for transgender rights is one of the strongest negative predictors of Biden favorability, indicating that cultural value alignment plays a major role for Independents.
- **Foreign-policy views** — Support for U.S. aid to Ukraine is a strong positive predictor, suggesting that geopolitical orientations meaningfully shape evaluations of the incumbent.
- **Economic and material concerns** — Inflation stress, border-security preferences, and abortion-related views show weak or statistically insignificant relationships with Biden support.

Although the analysis is cross-sectional and limited to a relatively small sample of self-identified Independents, the findings highlight that value-based and global issue orientations—not pocketbook or security anxieties—are central to how Independents evaluate President Biden.

## Discussion & Future Work

The results of this analysis raise important questions about how Independents form political attitudes and what issues truly drive their evaluations of presidential performance. The finding that cultural and foreign-policy concerns outweigh economic or border-security anxieties challenges common assumptions about voter behavior, especially in an election cycle where inflation and immigration dominate public discourse. One interpretation is that voters driven primarily by economic or security issues may have already sorted into stable partisan identities, leaving true Independents more responsive to value-based or global issues that signal broader ideological alignment.

Another possibility is that Independents who remain undecided tend to have relatively stable economic conditions, making symbolic or identity-related issues more salient than immediate material concerns. Alternatively, cultural and foreign-policy attitudes may serve as broader proxies for underlying worldviews—cosmopolitanism vs. nationalism, progressive vs. conservative identity—that shape how voters interpret the performance of political leaders.

#### **Future Work**

Several avenues for further research could extend and deepen these findings:

- **Expand the set of controls.** Including education, income, occupation, and especially party “leaners” would clarify whether the observed patterns persist when socioeconomic and ideological variables are incorporated.
- **Use longitudinal or panel data.** Tracking the same individuals over time would help determine whether cultural and foreign-policy concerns causally influence candidate evaluations, or whether shifting favorability leads to changes in issue attitudes.
- **Test different voter subgroups.** Comparing Independents to Democrats, Republicans, or “Independent leaners” could reveal whether the importance of cultural/global issues is unique to true Independents.
- **Incorporate additional political anxiety dimensions.** Issues such as climate change, crime, or health policy may provide more comprehensive measures of the anxiety landscape shaping Independent voter behavior.


 In summary, expanding the dataset, incorporating richer controls, and employing more robust causal methods would allow future research to more definitively determine why cultural and foreign-policy concerns play such an outsized role in shaping Independent voters’ evaluations of President Biden.

---

## 6. References
- **American National Election Studies.** (2024). *ANES 2024 Pilot Study*. Retrieved from https://electionstudies.org/data-center/2024-pilot-study/

- **White House.** (2022). *Fact Sheet: President Biden Announces Student Loan Relief for Borrowers Who Need It Most.* Biden White House Archives. https://bidenwhitehouse.archives.gov/briefing-room/statements-releases/2022/08/24/fact-sheet-president-biden-announces-student-loan-relief-for-borrowers-who-need-it-most/